# mRPI

**Marcelo Forets**, Universidad de la República, Uruguay.

----

References:

- [1] *Invariant Approximations of the Minimal Robust Positively Invariant Set*. S. V. Rakovic, E. C. Kerrigan, K. I. Kouramas, and D. Q. Mayne, https://ieeexplore.ieee.org/document/1406138

In the setting of [1], let

$$
x^+ = Ax + w
$$
be a discrete-time linear time-invariant system, where $x \in \mathbb{R}^n$ is the current state, $x^+$ is the succesor state, and $w \in \mathbb{R}^n$ is an unknown disturbance. $A \in \mathbb{R}^{n\times n}$ is assumed to be such that $\textrm{spec}(A) = \{\lambda \in \mathbb{C}: \lambda \textrm{ is an eigenvalue of } A\} \subsetneq D_2$ where $D_2$ is the unit disk in the complex plane. In addition, the disturbance $w$ is contained in a compact and convex set $W \subseteq \mathbb{R}^n$ that contains the origin.

The aim is to compute the set

$$
F_\infty = \bigoplus_{i=0}^\infty A^i W,
$$
where $A^0 := Id$. However, this set cannot be computed exactly in general, and one should consider either inner or outer approximations. For any $s \geq 0$, let
$$
F_s := \bigoplus_{i=0}^{s-1} A^i W,\qquad F_0 := \{ 0\}.
$$

## Preliminaries

To fix ideas consider the following example.

In [51]:
using LazySets

A = [-1/2 1.0; 0.0 1/2.0]

2×2 Array{Float64,2}:
 -0.5  1.0
  0.0  0.5

In [52]:
using LinearAlgebra

eigvals(A)

2-element Array{Float64,1}:
 -0.5
  0.5

In [53]:
all(norm(λ) < 1 for λ in eigvals(A)) # check that A is strictly stable

true

In [54]:
W = BallInf(zeros(2), 1.0)

BallInf{Float64}([0.0, 0.0], 1.0)

To illustrate, we can compute $F_4$ using LazySets as follows:

In [55]:
F(s) = MinkowskiSumArray([A^i * W for i in 0:s-1])
F4 = F(4)

MinkowskiSumArray{Float64,LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}}(LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}[LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([1.0 0.0; 0.0 1.0], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([-0.5 1.0; 0.0 0.5], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([0.25 0.0; 0.0 0.25], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([-0.125 0.25; 0.0 0.125], BallInf{Float64}([0.0, 0.0], 1.0))])

Here, $F_4$ has been *lazily* computed as the Minkowski sum array

$$
F_4 = W \oplus A W \oplus A^2 W \oplus A^3 W
$$

LazySets can be used to reason about $F_4$ without actually computing it. Underlying computations such as membership or inclusion checks, or overapproximations, can be performed using the theory of [support functions](https://en.wikipedia.org/wiki/Support_function).

**Note.** Although the n-ary lazy minkowski sum operation can be implemented with nested binary operations, for efficiency reasons, LazySets provides the binary operation `MinkowskiSum` as well as the n-ary opearation `MinkowskiSumArray`. A similar comment applies to other operations such as convex hulls and cartesian products. 

The support function of a convex polytope is a linear program, and the support function of a Minkowski sum is the sum of the support functions of each set. More properties of support functions and support vectors are found in the [LazySets documentation](https://juliareach.github.io/LazySets.jl/latest/man/polyhedral_approximations/).

For instance we can make the inclusion check $\alpha (1-\alpha)^{-1} \subseteq B_\inf^2(0.01)$ as follows:

In [66]:
α = 0.5
ε = 8.
α * (1-α)^-1 * F(2) ⊆ BallInf(zeros(2), ε) # Ballp is available as well, for any p >= 1

true

This computation has used support functions, as explained above (Julia provides the `@which` macro to know which function is actually used).

In [67]:
α * (1-α)^-1 * F(20) ⊆ BallInf(zeros(2), ε)

true

A final note: above we compute `F(s)` lazily: it is very efficient to create and ask a few evaluations of the support function. Here we compute up to $s = 1000$ very quickly:

In [68]:
[@time F(s) for s in 1:100:1000];

  0.000024 seconds (6 allocations: 320 bytes)
  0.000081 seconds (806 allocations: 80.781 KiB)
  0.000130 seconds (1.90 k allocations: 192.969 KiB)
  0.000204 seconds (3.09 k allocations: 316.484 KiB)
  0.000274 seconds (4.37 k allocations: 449.813 KiB)
  0.000348 seconds (5.73 k allocations: 590.953 KiB)
  0.000439 seconds (7.15 k allocations: 730.922 KiB)
  0.000704 seconds (8.66 k allocations: 880.078 KiB)
  0.001076 seconds (10.21 k allocations: 1.009 MiB)
  0.001458 seconds (11.79 k allocations: 1.162 MiB)


In applications it may be also necessary to use a *concrete* set representation. Actually a combination of concrete and lazy should also be considered, see for example the discussion in [Reach Set Approximation through Decomposition with Low-dimensional Sets and High-dimensional Matrices. Sergiy Bogomolov, Marcelo Forets, Goran Frehse, Andreas Podelski, Christian Schilling, Frédéric Viry](https://arxiv.org/abs/1801.09526).

In LazySets, concrete set operations are denoted with lower case letters. 

In [69]:
using Polyhedra # used for most concrete set operations

F1 = W
F2 = minkowski_sum(F1, linear_map(A, W))
F3 = minkowski_sum(F2, linear_map(A, W))
F4 = minkowski_sum(F3, linear_map(A, W))

VPolytope{Float64}(Array{Float64,1}[[-5.5, -2.5], [-0.5, -2.5], [5.5, 0.5], [5.5, 2.5], [0.5, 2.5], [-5.5, -0.5]])

Here `F4` is a *concrete* set, a polytope in vertex representation. A polytope in half-space representation, for instance, is also said to be a concrete set.

## Implementation of Algorithm 1

An implementation of Algorithm 1 in [1] using LazySets. is presented below. There is room for micro-optimizations, but the code was kept simple and generic on purpose.

The key part of the algorithm is to handle the following functions:
$$
s^0(\alpha) := \min \{ s \in \mathbb{N}_+ : A^s W \subseteq \alpha W \} \\
\alpha^0(\alpha) := \min \{ \alpha \in \mathbb{R} : A^s W \subseteq \alpha W \}
$$

If it is assumed that $W$ is a polytope in constraint representation, the inclusion checks can be easily tested using support functions as explained in the paper. Actually this can be generalized substantially: if the support function of $W$ is available,computations with support functions also apply (possibly with an overapproximation error).

To fix ideas we consider the following example.

In [95]:
# EXAMPLE: inputs are A, W, ε > 0
A = [-1/2 1.0; 0.0 1/2.0]
W = BallInf(zeros(2), 1.0)

BallInf{Float64}([0.0, 0.0], 1.0)

In [96]:
# cf. Eq. (11) in [1]
function α₀(s, A, W)
    F, g = tosimplehrep(W) # W = {x s.t. Fx <= g}
    return maximum(ρ(transpose(A^s)*F[i, :], W) / g[i] for i in eachindex(g))
end

α₀ (generic function with 1 method)

In [97]:
α₀(2, A, W), α₀(5, A, W), α₀(10, A, W)

(0.25, 0.09375, 0.0009765625)

In [98]:
using LazySets.Arrays

# cf. Eq. (13) in [1]
function M(s, A, W)
    n = LazySets.dim(W)
    max_j = Vector{Float64}()
    for j in 1:n
        ej = SingleEntryVector(j, n, 1.0)
        M⁺ = M_plus = sum(ρ(transpose(A^i) * ej, W) for i in 0:s-1)
        M⁻ = sum(ρ(transpose(A^i) * (-ej), W) for i in 0:s-1)
        push!(max_j, max(M⁺, M⁻))
    end
    return maximum(max_j)
end

M (generic function with 1 method)

In [99]:
M(2, A, W), M(3, A, W), M(4, A, W)

(2.5, 2.75, 3.125)

In [119]:
# Implementation
function mRPI(A, W, ε)
    smax = 1000
    s = 0
    while true
        s += 1
        α = α₀(s, A, W)
        Ms = M(s, A, W)
        s == smax && error("convergence not achieved for s = $smax")
        if α <= ε/(ε+Ms)
            break
        end
    end
    return s
end

mRPI (generic function with 1 method)

In [120]:
s = mRPI(A, W, 1e-1)

6

In [121]:
s = mRPI(A, W, 1e-2)

10

In [122]:
s = mRPI(A, W, 1e-5)

20

## Computing $F(\alpha, s)$ as a lazy set

It verififes that to have better accuracy, the required `s` increases.

The set $F(\alpha, s)$ is computed lazily as follows.

In [127]:
# compute Fs lazily
ε = 1e-2
s = mRPI(A, W, ε)
Fs = MinkowskiSumArray([A^i * W for i in 0:s-1])
Fαs = 1/(1-α) * Fs

LinearMap{Float64,MinkowskiSumArray{Float64,LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}},Float64,SparseMatrixCSC{Float64,Int64}}(
  [1, 1]  =  2.0
  [2, 2]  =  2.0, MinkowskiSumArray{Float64,LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}}(LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}[LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([1.0 0.0; 0.0 1.0], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([-0.5 1.0; 0.0 0.5], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([0.25 0.0; 0.0 0.25], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([-0.125 0.25; 0.0 0.125], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([0.0625 0.0; 0.0 0.0625], BallInf{Float64}([0.0, 0.0], 1.0)), LinearMap{Float64,BallInf{Float64},Float64,Array{Float64,2}}([-0.0312

This is an interesting representation because if we only want to query some directions of the approximatio of the mRPI, it can be done with `Fαs` and there is no need to overapproximate it. For instance, if we'd like to compute the maximum distance to the origin along the direction `[1, 0]`, this is:

In [136]:
ρ([1.0, 0.0], Fαs) # the support function of a lazy set does not require Fαs to be concrete

6.66015625

Similarly, if we are interested in the vertex of `Fαs` which is further from the origin along direction `[1, 0]` this is:

In [137]:
σ([1.0, 0.0], Fαs) # the support vector of a lazy set does not require Fαs to be concrete

2-element Array{Float64,1}:
 6.66015625
 3.99609375

## Computing $F(\alpha, s)$ using iterative refinement

Since this set is two-dimensional, we can use iterative refinement with tolerance $\varepsilon$ and compute an overapproximation $\hat{F}(\alpha, s)$ of $F(\alpha, s)$ within Hausdorff distance $\varepsilon$.

Iterative refinement in higher dimensions is not implemented yet (see [LazySets#969](https://github.com/JuliaReach/LazySets.jl/issues/969)).

In this approach, we begin with the lazily computed set `Fαs` and then find the constraints that define it, by succseively querying support functions.

In [139]:
# overapproximate Fαs
using LazySets.Approximations

# a concrete polygon in constraint representation
F̂αs = overapproximate(Fαs, ε)

HPolygon{Float64}(LazySets.HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,Array{Float64,1}}([1.0, 0.0], 6.66016), HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0], 3.99609), HalfSpace{Float64,Array{Float64,1}}([-0.242536, 0.970143], 4.19985), HalfSpace{Float64,Array{Float64,1}}([-0.447214, 0.894427], 4.16992), HalfSpace{Float64,Array{Float64,1}}([-1.0, 0.0], 6.66016), HalfSpace{Float64,Array{Float64,1}}([0.0, -1.0], 3.99609), HalfSpace{Float64,Array{Float64,1}}([0.316228, -0.948683], 4.21225), HalfSpace{Float64,Array{Float64,1}}([0.447214, -0.894427], 4.16992)])

It is guaranteed that the Hausdorff distance between `F̂αs` and the exact `Fαs` is no bigger than `ε`. See the [Iterative Refinement](https://juliareach.github.io/LazySets.jl/latest/man/iterative_refinement/) section in LazySets documentation or the source code docstrings `src/Approximations/iterative_refinement.jl` for more details.

## Computing $F(\alpha, s)$ without iterative refinement

One can use support functions to overapproximate any convex set, by defining which directions should be evaluated. For instance, we can sample the unit sphere in `n`-dimensions and evaluate the support function of `Fαs` along all those randomly selected directions. The polytope obtained is an overapproximation of `Fαs`.

This approach is similar to the last one and has the advantage that *it is not limited to two-dimensions.* The disadvantage is that we don't control the error of the approximation.

In [146]:
# we overload overapproximate for this application so that it works with any array of directions 
function LazySets.Approximations.overapproximate(X::LazySet{N}, dirs::Vector{Vector{N}}) where {N}
    halfspaces = Vector{LinearConstraint{N}}()
    sizehint!(halfspaces, length(dirs))
    H = HPolytope(halfspaces)
    for d in dirs
        addconstraint!(H, LinearConstraint(d, ρ(d, X)))
    end
    return H
end

using Distributions
using LazySets: _sample_unit_nsphere_muller!

# refine the set X using the given number of samples
function refine(X, nsamples)
    n = LazySets.dim(X)
    dirs = Vector{Vector{Float64}}(undef, nsamples)
    _sample_unit_nsphere_muller!(dirs, n, nsamples)
    return overapproximate(X, dirs)
end

refine (generic function with 1 method)

In [148]:
F̂αs_random_refinement = refine(Fαs, 10)

HPolytope{Float64}(LazySets.HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,Array{Float64,1}}([-0.999888, -0.0149501], 6.71915), HalfSpace{Float64,Array{Float64,1}}([-0.99809, -0.0617811], 6.89432), HalfSpace{Float64,Array{Float64,1}}([-0.0793736, 0.996845], 4.08921), HalfSpace{Float64,Array{Float64,1}}([0.983874, -0.178861], 6.79101), HalfSpace{Float64,Array{Float64,1}}([-0.717238, -0.696829], 7.56151), HalfSpace{Float64,Array{Float64,1}}([0.269279, 0.963062], 5.64193), HalfSpace{Float64,Array{Float64,1}}([0.257841, -0.966187], 4.20443), HalfSpace{Float64,Array{Float64,1}}([-0.533095, 0.846055], 4.67747), HalfSpace{Float64,Array{Float64,1}}([-0.974184, -0.225757], 7.39036), HalfSpace{Float64,Array{Float64,1}}([-0.999791, 0.0204633], 6.68602)])

## Computing $F(\alpha, s)$ with concrete operations: HPolytope

We can otherwise resort to *concrete* intermediate operations: instead of building `Fαs` lazily, we build it using concrete linear map and concrete minkowski sum. This approach is thus limited by the efficiency with which these operations are defined.

Let's focus on `HPolytope`'s (as in [1]). The "challenge" is to make the intermediate computations in H-representation.

- The linear map can be computed without passing to the V-rep provided that the coefficients matrix is invertible. See `linear_map(::AbstractPolyhedron)` in LazySets for implementation details and available options.
- Run Appendix 1 below to get `minkowski_sum_hrep` (or [LazySets#1508](https://github.com/JuliaReach/LazySets.jl/pull/1508)) for an implementation using LazySets, Polyhedra and CDDLib.

In [164]:
using CDDLib

F1 = W
F2 = minkowski_sum_hrep(F1, linear_map(A, W))
F3 = minkowski_sum_hrep(F2, linear_map(A^2, W))
F4 = minkowski_sum_hrep(F3, linear_map(A^3, W))

HPolytope{Float64}(LazySets.HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,Array{Float64,1}}([0.0, -1.0], 1.875), HalfSpace{Float64,Array{Float64,1}}([1.0, -2.0], 4.375), HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0], 1.875), HalfSpace{Float64,Array{Float64,1}}([-1.0, 2.0], 4.375), HalfSpace{Float64,Array{Float64,1}}([-1.0, -0.0], 3.125), HalfSpace{Float64,Array{Float64,1}}([1.0, -0.0], 3.125)])

**Note.** You can also use `minkowski_sum` here but it will make the computation in vertex representation. For two-dimensions an efficient implementation is available, so it may be worth considering at least a three dimensional example.

## Computing $F(\alpha, s)$ with concrete operations: Zonotope

Zonotopes have been studied in the context of reachability analysis because they behave well with respect to linear maps and minkowski sums (in the sense that they are closed under these operations, and can be performed efficiently).

Instead of building $F(\alpha, s)$ as a lazy set and then approximating with a polytope, we can sequentially build $F(\alpha, s)$ using concrete operations with zonotopes. 

We first need to convert the input set, $W$, to a zonotope, then use concrete set operations; Julia's multiple-dispatch, is such that `minkowski_sum(::Zonotope, ::Zonotope)` uses the appropriate, efficient method for zonotopes.

In [168]:
Wzono = convert(Zonotope, W)

Zonotope{Float64}([0.0, 0.0], [1.0 0.0; 0.0 1.0])

In [171]:
F1 = W
F2 = minkowski_sum(F1, linear_map(A, Wzono))
F3 = minkowski_sum(F2, linear_map(A^2, Wzono))
F4 = minkowski_sum(F3, linear_map(A^3, Wzono))

Zonotope{Float64}([0.0, 0.0], [1.0 0.0 … -0.125 0.25; 0.0 1.0 … 0.0 0.125])

In [173]:
order(F4) # the zonotope order is the number of generators divided by the dimension

4//1

It is worth noting that this approach is not limited to low dimensions. The sole limitation is that the accumulation of nested Minkowski sums will make the number of generators to increase linearly in `s`. The typical strategy is to reduce the number of generators obtaining an overapproximating zonotope with less generators. In LazySets, the `reduce` function can be used to reduce the number of generators, or simply `overapproximate(::Zonotope, Zonotope, ord)` where `ord` is the target order. I'm not aware of an overapproximating scheme *with guaranteed Hausdorff error* (as with the iterative refinement method).

In [177]:
reduce_order(F4, 2)

Zonotope{Float64}([0.0, 0.0], 
  [1, 1]  =  0.25
  [2, 1]  =  0.125
  [1, 2]  =  1.0
  [2, 2]  =  0.5
  [1, 3]  =  1.875
  [2, 4]  =  1.25)

## Appendix: Minkowski sum in H-representation

In [149]:
# see LazySets#1508
function minkowski_sum_hrep(P::LazySet{N}, Q::LazySet{N};
                            backend=nothing,
                            algorithm=nothing,
                            prune=true) where {N<:Real}

    if backend == nothing
        if N <: Rational
            backend = CDDLib.Library(:Exact)
        else
            backend = CDDLib.Library()
        end
    end
    if algorithm == nothing
        algorithm = Polyhedra.FourierMotzkin()
    end
    
    A, b = tosimplehrep(P) # works as long as P has a constraints_list implementation
    C, d = tosimplehrep(Q) # similarly for Q
    mP, nP = size(A)
    mQ, nQ = size(C)
    E = [zeros(N, mP, nQ) A; C -C]
    f = [b; d]
    PQ = HPolytope(E, f)
    PQ_cdd = polyhedron(PQ, backend=backend)
    W = Polyhedra.eliminate(PQ_cdd, nP+1:2nP, algorithm)
    W = HPolytope(W)
    if prune
        remove_redundant_constraints!(W)
    end
    return W
end

minkowski_sum_hrep (generic function with 2 methods)